In [3]:
# !pip install GitPython

In [1]:
import os
import openai
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Insert your OpenAI API Key: ")

Insert your OpenAI API Key: ········


# Repositório Local

In [2]:
from git import Repo, GitCommandError


try:
    repo = Repo.clone_from(
        "https://github.com/hwchase17/langchain", to_path="./data/test_repo1"
    )
except GitCommandError:
    pass

repo = Repo("./data/test_repo1")
    
branch = repo.head.reference


In [3]:
from langchain.document_loaders import GitLoader
loader = GitLoader(repo_path="./data/test_repo1/", branch=branch, )
documents = loader.load()

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, 
    chunk_overlap=50, 
    separators=["\n\n", "\n", " ", ""]
)
texts = text_splitter.split_documents(documents)

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="clips/mfaq")

/home/eduardo/anaconda3/lib/python3.10/site-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [6]:
from langchain.vectorstores import FAISS

# -- demora ----------------------------
vector_store = FAISS.from_documents(texts, embeddings)
vector_store.save_local("data/git-huggingface.db")

In [7]:
%%time
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="clips/mfaq")
vector_store = FAISS.load_local("data/git-huggingface.db", embeddings)

/home/eduardo/anaconda3/lib/python3.10/site-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


CPU times: user 1.69 s, sys: 544 ms, total: 2.23 s
Wall time: 1.85 s


In [8]:
from langchain.chains  import RetrievalQA
from langchain.llms import OpenAI

# Escolha o modelo de lingaugem
llm = OpenAI(model='text-davinci-003', temperature=0)

# Instancie o retriever
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever())

In [9]:
qa.run("Como é definido o prompt para perguntas e respostas?")

' O prompt para perguntas e respostas é definido usando o PromptTemplate, que é responsável por construir um valor de prompt. Estes PromptTemplates podem fazer coisas como formatação, seleção de exemplos e mais. Em um nível alto, esses são basicamente objetos que expõem um método `format_prompt` para construir um prompt. No nível interno, QUALQUER coisa pode acontecer.'

In [10]:
qa.run("Qual é o arquivo que tenho que alterar para customizar o prompt de ConversationalRetrievalChain?")

' Você precisa alterar o arquivo "conversational_retrieval_chain.py" para customizar o prompt de ConversationalRetrievalChain.'

In [11]:
qa.run("Qual é o caminho do arquivo conversational_retrieval_chain.py?")

' O arquivo conversational_retrieval_chain.py está localizado na pasta langchain/chains/conversational_retrieval_chain.py.'

# ConversationalRetrievalChain

In [14]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [15]:
from langchain.chains import ConversationalRetrievalChain

llm = OpenAI(temperature=0)

qa = ConversationalRetrievalChain.from_llm(
    llm,
    vector_store.as_retriever(),
    memory=memory
)

In [17]:
query = "Qual é o arquivo que tenho que alterar para customizar o prompt de ConversationalRetrievalChain?"
result = qa({"question": query})
result["answer"]

' Você precisa alterar o arquivo de configuração do ConversationalRetrievalChain para customizar o prompt.'

In [18]:
query = "qual é o caminho desse arquivo?"
result = qa({"question": query})
result["answer"]

' Não sei.'

In [19]:
memory.clear()

In [20]:
query = "Qual é o arquivo que tenho que alterar para customizar o prompt da classe ConversationalRetrievalChain?"
result = qa({"question": query})
result["answer"]

' O arquivo que você precisa alterar para customizar o prompt da classe ConversationalRetrievalChain é o langchain/prompts.py.'

In [21]:
query = "Esse arquivo existe?"
result = qa({"question": query})
result["answer"]

" I don't know."

In [23]:
query = "Eu verifiquei e ele não existe. Pode me dar uma resposta alternativa?"
result = qa({"question": query})
result["answer"]

' Você pode usar o arquivo langchain/prompts/prompt.py para customizar o prompt da classe ConversationalRetrievalChain.'